In [1]:
println("Hello World!")

# Assign the current working directory path to a variable
current_directory = pwd()

# String concatenation is accomplished using commas
println("Currently, this notebook is running in the directory ", current_directory)

Hello World!
Currently, this notebook is running in the directory /Users/xronos/julia-for-data-science


In [ ]:
# To install a package, run the following line in this notebook or from the Julia REPL.
# import Pkg; Pkg.add("CSV")

In [3]:
# If the CSV package is already installed, this is the only line necessary to use it in a program.
using CSV 

In [4]:
# Let's pull in some data for analysis
# This file is from https://archive.ics.uci.edu/ml/datasets/magic+gamma+telescope
data = CSV.File("./data/Magic/magic04.csv") 

print(data)

CSV.File("./data/Magic/magic04.csv"):
Size: 19019 x 11
Tables.Schema:
 Symbol("28.7967")  Float64
 Symbol("16.0021")  Float64
 Symbol("2.6449")   Float64
 Symbol("0.3918")   Float64
 Symbol("0.1982")   Float64
 Symbol("27.7004")  Float64
 Symbol("22.011")   Float64
 Symbol("-8.2027")  Float64
 Symbol("40.092")   Float64
 Symbol("81.8828")  Float64
 :g                 String 

We'll use the `magic` dataset and classify the observed radiation of the telescope as `gamma` or `hadron`.

## k-Nearest Neighbors Classification
The diagram of the kNN algorithm is below.

<img src="./kNN-diagram.jpeg" alt="kNN diagram">

In [9]:
# Distance should take in two, 1-dimensional vectors and return a single number (float).
# The commented-out implementation specifies input data types and the return data type
# function distance(x::Array{Float64, 1}, y::Array{Float64, 1})::Float64
#     distance = 0
#     for in in 1:length(x)
#         dist += (x[i] - y[i])^2
#     end
#     distance = sqrt(distance)
#     return distance
# end

function distance(x, y, distance_measure="euclidean")
    if distance_measure == "city_block_distance"
        return city_blocks(x, y)
    end
    return euclidean_distance(x, y)
end

function euclidean_distance(x, y)
    distance = 0
    for i in 1:length(x)
        distance += (x[i] - y[i])^2
    end
    distance = sqrt(distance)
    return distance
end

function city_block_distance(x, y)
    distance = 0
    for i in 1:length(x)
        distance += abs(x[i] - y[i])
    end
    distance = sqrt(distance)
    return distance
end
    

manhattan_distance (generic function with 1 method)

In [3]:
# Define the implementation of the classify function
function classify(distances, labels, k)
    # Find all of the distinct classes
    class = unique(labels)

    # Determine the number of classees
    number_of_classes = length(class) 
    
    # Initialize the vector of indexes
    indexes = Array{Int, 0}
    
    # Specify the largest possible number that this vector can have
    M = typemax(distances[1])
    
    class_count = Array{Int, number_of_classes}
    for i in 1:k
        indexes[i] = inmin(distances)
        
        # make sure this element is not selected again
        distances[indexes[i]] = M
    end
    
    klabels = labels[indexes]
    for i in 1:number_of_classes
        for j in 1:k
            if klabels[j] == class[i]
                class_count[i] += 1
                break
            end
        end
    end
    index = inmax(class_count)
    return class[index]
end


classify (generic function with 1 method)

In [44]:
# Another auxiliary function of the implementation of the kNN algorithm. This one performs classification of a point based on its distances from the known points of the dataset.
function apply_kNN(X, x, Y, k)

    N = size(X, 1) # Number of known datapoints
    n = size(Y, 1) # number of datapoints to classify
    D = Array{Float64}(undef, N)
    z = Array{typeof(x[1]), n} # Initialize the labels vector (output)
    for i in 1:n
        for j in 1:N
            D[j] = distance(X[j,:], Y[i,:])
        end
        z[i] = classify(D, x, k)
    end
    return z
end
    

apply_kNN (generic function with 1 method)

In [45]:
print(data)

19019×11 DataFrame
│ Row   │ 28.7967 │ 16.0021 │ 2.6449  │ 0.3918  │ 0.1982  │ 27.7004  │ 22.011   │ -8.2027  │ 40.092  │ 81.8828 │ g      │
│       │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64  │ Float64  │ Float64  │ Float64 │ Float64 │ String │
├───────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┼──────────┼──────────┼─────────┼─────────┼────────┤
│ 1     │ 31.6036 │ 11.7235 │ 2.5185  │ 0.5303  │ 0.3773  │ 26.2722  │ 23.8238  │ -9.9574  │ 6.3609  │ 205.261 │ g      │
│ 2     │ 162.052 │ 136.031 │ 4.0612  │ 0.0374  │ 0.0187  │ 116.741  │ -64.858  │ -45.216  │ 76.96   │ 256.788 │ g      │
│ 3     │ 23.8172 │ 9.5728  │ 2.3385  │ 0.6147  │ 0.3922  │ 27.2107  │ -6.4633  │ -7.1513  │ 10.449  │ 116.737 │ g      │
│ 4     │ 75.1362 │ 30.9205 │ 3.1611  │ 0.3168  │ 0.1832  │ -5.5277  │ 28.5525  │ 21.8393  │ 4.648   │ 356.462 │ g      │
│ 5     │ 51.624  │ 21.1502 │ 2.9085  │ 0.242   │ 0.134   │ 50.8761  │ 43.1887  │ 9.8145   │ 3.613   │ 238.098 │ g      │
│ 6  

│ 134   │ 18.8734 │ 17.772  │ 2.5821  │ 0.4686  │ 0.2579  │ -14.6576 │ -16.9343 │ -14.4939 │ 17.1286 │ 114.938 │ g      │
│ 135   │ 34.1619 │ 21.2966 │ 2.9974  │ 0.2907  │ 0.1554  │ -4.6579  │ -11.6112 │ 11.8667  │ 11.6079 │ 163.722 │ g      │
│ 136   │ 30.0005 │ 21.6966 │ 2.934   │ 0.3062  │ 0.1857  │ 23.3297  │ 17.6318  │ -18.5123 │ 27.943  │ 151.522 │ g      │
│ 137   │ 41.3848 │ 32.4617 │ 2.7938  │ 0.3521  │ 0.1889  │ -20.4985 │ 28.2927  │ 28.1989  │ 9.628   │ 308.373 │ g      │
│ 138   │ 25.2228 │ 13.6075 │ 2.3738  │ 0.4989  │ 0.2939  │ -12.181  │ -12.8844 │ 11.3032  │ 66.649  │ 78.9436 │ g      │
│ 139   │ 21.6891 │ 10.3849 │ 2.316   │ 0.5604  │ 0.2923  │ 5.8526   │ -17.6826 │ 10.2235  │ 1.5485  │ 232.154 │ g      │
│ 140   │ 20.5839 │ 12.8876 │ 2.4191  │ 0.4724  │ 0.2648  │ 12.5054  │ 12.7741  │ -8.1001  │ 26.586  │ 199.96  │ g      │
│ 141   │ 25.2091 │ 10.6551 │ 2.3701  │ 0.4989  │ 0.3006  │ 10.6071  │ 21.1038  │ 3.9151   │ 41.563  │ 147.489 │ g      │
│ 142   │ 35.7062 │ 13.2

│ 270   │ 13.123  │ 10.9616 │ 2.1569  │ 0.7666  │ 0.4146  │ 14.6416  │ -4.0114  │ 11.6445  │ 88.1516 │ 98.8626 │ g      │
│ 271   │ 53.5736 │ 25.803  │ 3.5737  │ 0.1956  │ 0.0986  │ 39.2808  │ 58.4267  │ 14.9922  │ 7.596   │ 171.407 │ g      │
│ 272   │ 29.1123 │ 18.0166 │ 2.635   │ 0.3175  │ 0.1657  │ 11.2887  │ 26.7753  │ -10.7571 │ 36.953  │ 190.445 │ g      │
│ 273   │ 74.4746 │ 13.4246 │ 2.9717  │ 0.2818  │ 0.1435  │ 18.4636  │ 76.5354  │ -9.3669  │ 5.847   │ 286.639 │ g      │
│ 274   │ 29.4259 │ 17.0431 │ 2.4526  │ 0.3633  │ 0.2028  │ -14.6546 │ -15.5914 │ 8.1478   │ 45.059  │ 142.487 │ g      │
│ 275   │ 40.749  │ 26.1492 │ 3.2122  │ 0.1933  │ 0.1009  │ 29.9765  │ -24.5987 │ 8.9177   │ 8.364   │ 169.123 │ g      │
│ 276   │ 33.8898 │ 17.6924 │ 2.9647  │ 0.3026  │ 0.1719  │ 13.8792  │ 25.8841  │ 7.6886   │ 6.891   │ 90.7788 │ g      │
│ 277   │ 85.6478 │ 20.1038 │ 3.3537  │ 0.2719  │ 0.2004  │ -43.2684 │ 61.6568  │ -19.4233 │ 0.7472  │ 318.342 │ g      │
│ 278   │ 21.4944 │ 10.4

│ 339   │ 47.1637 │ 19.347  │ 2.9363  │ 0.2606  │ 0.143   │ 30.0344  │ 40.5783  │ -6.9531  │ 7.132   │ 180.522 │ g      │
│ 340   │ 70.6596 │ 30.9809 │ 3.1274  │ 0.1529  │ 0.0764  │ -77.6508 │ 48.7238  │ 12.3933  │ 11.138  │ 253.993 │ g      │
│ 341   │ 66.5662 │ 28.6046 │ 3.5894  │ 0.1552  │ 0.0779  │ 35.167   │ 40.3408  │ 20.895   │ 0.4     │ 261.59  │ g      │
│ 342   │ 28.3371 │ 24.9896 │ 2.968   │ 0.3337  │ 0.19    │ 27.1233  │ 22.0479  │ -15.7731 │ 7.208   │ 107.15  │ g      │
│ 343   │ 53.7088 │ 16.6628 │ 2.8695  │ 0.2984  │ 0.1668  │ 20.2522  │ 53.7607  │ -12.9033 │ 11.677  │ 252.365 │ g      │
│ 344   │ 79.7124 │ 35.2208 │ 3.3244  │ 0.2914  │ 0.1708  │ 10.1771  │ -28.5291 │ -18.2423 │ 2.129   │ 340.809 │ g      │
│ 345   │ 30.251  │ 13.6492 │ 2.6946  │ 0.3495  │ 0.1768  │ -1.3097  │ 17.8866  │ -9.1995  │ 2.103   │ 154.642 │ g      │
│ 346   │ 20.7708 │ 16.7921 │ 2.5815  │ 0.4404  │ 0.2267  │ 6.1544   │ 11.5784  │ 14.5851  │ 72.2425 │ 228.28  │ g      │
│ 347   │ 27.1848 │ 14.0

│ 448   │ 26.889  │ 9.8236  │ 2.4433  │ 0.4468  │ 0.2324  │ 34.4309  │ 19.4041  │ -6.9568  │ 18.419  │ 192.16  │ g      │
│ 449   │ 77.8141 │ 26.5122 │ 3.7987  │ 0.1644  │ 0.0897  │ 37.0051  │ 65.7207  │ 14.2353  │ 2.712   │ 218.662 │ g      │
│ 450   │ 20.2467 │ 10.4762 │ 2.3096  │ 0.6275  │ 0.3162  │ 8.6645   │ -18.0061 │ 10.9604  │ 79.4844 │ 161.749 │ g      │
│ 451   │ 19.8657 │ 10.773  │ 2.3324  │ 0.5953  │ 0.3326  │ -8.0461  │ 17.5656  │ 9.9813   │ 17.929  │ 53.4503 │ g      │
│ 452   │ 18.4491 │ 7.3587  │ 2.1945  │ 0.8051  │ 0.4824  │ 3.9625   │ -8.2376  │ 2.9172   │ 42.315  │ 135.144 │ g      │
│ 453   │ 39.4796 │ 16.8977 │ 2.932   │ 0.3474  │ 0.1959  │ 31.9808  │ 28.3     │ 6.6446   │ 1.129   │ 198.882 │ g      │
│ 454   │ 112.635 │ 31.147  │ 2.9868  │ 0.1918  │ 0.1139  │ -117.267 │ 89.9887  │ 23.0827  │ 17.724  │ 277.313 │ g      │
│ 455   │ 27.1373 │ 13.7348 │ 2.4281  │ 0.4851  │ 0.2743  │ 6.3929   │ 20.5919  │ 9.8976   │ 17.2732 │ 157.231 │ g      │
│ 456   │ 73.2648 │ 27.0

...

│ 3306  │ 68.3934 │ 23.161  │ 2.8109  │ 0.3725  │ 0.1924  │ -54.154  │ -22.6559 │ 18.5595  │ 7.9012  │ 309.048 │ g      │
│ 3307  │ 85.2939 │ 30.9097 │ 3.6512  │ 0.1237  │ 0.0619  │ 62.5144  │ 74.9125  │ -17.053  │ 2.4232  │ 229.733 │ g      │
│ 3308  │ 59.5126 │ 31.0478 │ 3.2827  │ 0.1731  │ 0.1051  │ 58.4711  │ 61.3665  │ 10.245   │ 1.5087  │ 86.7844 │ g      │
│ 3309  │ 18.0933 │ 15.3799 │ 2.2978  │ 0.5441  │ 0.335   │ -5.8481  │ 15.9569  │ -10.4222 │ 82.7585 │ 137.232 │ g      │
│ 3310  │ 21.4177 │ 6.6608  │ 2.1123  │ 0.7413  │ 0.4595  │ 20.9604  │ 15.1482  │ -6.3638  │ 38.6953 │ 121.457 │ g      │
│ 3311  │ 54.6903 │ 13.3808 │ 2.6498  │ 0.3225  │ 0.1691  │ 9.216    │ 41.7832  │ 8.7147   │ 22.307  │ 84.5171 │ g      │
│ 3312  │ 45.7388 │ 14.8798 │ 3.1654  │ 0.2924  │ 0.1554  │ 42.1786  │ 32.4473  │ 9.2524   │ 5.0177  │ 173.518 │ g      │
│ 3313  │ 91.9596 │ 25.8895 │ 3.6091  │ 0.1473  │ 0.0781  │ 49.0344  │ 97.9016  │ 17.1415  │ 7.4474  │ 233.582 │ g      │
│ 3314  │ 19.1712 │ 16.2

│ 3374  │ 54.323  │ 21.3628 │ 3.3105  │ 0.1722  │ 0.0873  │ -93.2578 │ 20.9643  │ -6.5174  │ 6.759   │ 270.21  │ g      │
│ 3375  │ 31.9543 │ 15.3539 │ 2.6604  │ 0.3213  │ 0.1781  │ 14.7364  │ 19.2773  │ -6.4606  │ 2.124   │ 150.524 │ g      │
│ 3376  │ 71.3588 │ 34.9759 │ 3.5158  │ 0.1845  │ 0.102   │ 19.2408  │ 70.827   │ 23.5493  │ 1.576   │ 289.206 │ g      │
│ 3377  │ 80.393  │ 24.7088 │ 3.1874  │ 0.317   │ 0.1738  │ 0.9793   │ 43.4367  │ -14.4227 │ 0.3113  │ 341.943 │ g      │
│ 3378  │ 60.8649 │ 14.4934 │ 2.8689  │ 0.307   │ 0.1684  │ -48.7893 │ 23.8006  │ 10.5276  │ 4.081   │ 184.962 │ g      │
│ 3379  │ 36.3933 │ 14.9178 │ 2.9292  │ 0.3308  │ 0.176   │ 46.1626  │ 29.8447  │ -11.6154 │ 2.6313  │ 177.111 │ g      │
│ 3380  │ 19.9697 │ 12.0635 │ 2.2989  │ 0.6533  │ 0.3693  │ 8.9327   │ 20.5025  │ 7.2398   │ 2.681   │ 96.9348 │ g      │
│ 3381  │ 23.9923 │ 22.4191 │ 2.6069  │ 0.2818  │ 0.1545  │ -14.6134 │ 16.1367  │ -13.09   │ 44.853  │ 151.049 │ g      │
│ 3382  │ 59.4502 │ 18.2

│ 3443  │ 52.9929 │ 17.9574 │ 2.8579  │ 0.3689  │ 0.2004  │ -36.8339 │ 40.5554  │ -16.2243 │ 4.885   │ 294.535 │ g      │
│ 3444  │ 45.8448 │ 25.5516 │ 3.1402  │ 0.1926  │ 0.1162  │ 69.452   │ 23.7428  │ 12.6999  │ 0.8053  │ 189.467 │ g      │
│ 3445  │ 60.4087 │ 15.2666 │ 2.9175  │ 0.2382  │ 0.1276  │ 67.2909  │ 40.1999  │ 6.3825   │ 2.6457  │ 232.763 │ g      │
│ 3446  │ 20.9316 │ 15.2379 │ 2.4857  │ 0.4575  │ 0.2402  │ 25.2371  │ 14.1903  │ 5.9376   │ 36.675  │ 137.007 │ g      │
│ 3447  │ 100.012 │ 44.7902 │ 4.1322  │ 0.0884  │ 0.0453  │ 43.2948  │ 89.1063  │ -26.9751 │ 3.081   │ 252.784 │ g      │
│ 3448  │ 38.9342 │ 20.5513 │ 3.1133  │ 0.2519  │ 0.1498  │ 2.8118   │ -11.6282 │ -15.5647 │ 5.5697  │ 170.239 │ g      │
│ 3449  │ 50.7355 │ 13.8768 │ 2.4886  │ 0.3279  │ 0.1867  │ 22.9626  │ 32.073   │ -10.6266 │ 25.98   │ 165.046 │ g      │
│ 3450  │ 22.1148 │ 11.2098 │ 2.2068  │ 0.5652  │ 0.2888  │ 12.3217  │ 19.5864  │ 9.1503   │ 72.274  │ 201.773 │ g      │
│ 3451  │ 35.2851 │ 18.8

Excessive output truncated after 524292 bytes.

.7256 │ 19.9475 │ 2.8293  │ 0.2444  │ 0.1267  │ -77.8216 │ 28.2398  │ -8.4376  │ 3.973   │ 263.894 │ g      │
│ 3514  │ 31.1396 │ 24.7569 │ 2.7642  │ 0.2737  │ 0.1403  │ -53.2459 │ 17.078   │ 21.3696  │ 4.6106  │ 213.971 │ g      │
│ 3515  │ 57.0652 │ 24.0205 │ 3.745   │ 0.1781  │ 0.0929  │ 40.7773  │ 46.2678  │ -15.1825 │ 1.963   │ 199.619 │ g      │
│ 3516  │ 21.2721 │ 10.4919 │ 2.3802  │ 0.625   │ 0.3979  │ 19.36    │ 20.1849  │ 11.552   │ 38.5604 │ 68.1625 │ g      │
│ 3517  │ 27.2053 │ 14.347  │ 2.635   │ 0.4195  │ 0.2329  │ -0.5521  │ 21.9019  │ -8.3026  │ 1.8909  │ 215.623 │ g      │
│ 3518  │ 39.196  │ 15.1294 │ 2.5051  │ 0.3125  │ 0.1641  │ 10.1254  │ 13.5262  │ -5.4373  │ 14.662  │ 228.967 │ g      │
│ 3519  │ 49.9884 │ 18.3477 │ 2.9983  │ 0.2018  │ 0.1019  │ -6.3434  │ 28.8756  │ 11.7644  │ 4.2187  │ 187.793 │ g      │
│ 3520  │ 35.4716 │ 17.892  │ 2.7551  │ 0.2988  │ 0.1573  │ 7.156    │ -24.0317 │ -10.3888 │ 20.1401 │ 167.218 │ g      │
│ 3521  │ 34.8702 │ 18.7861 │ 2.639 

In [46]:
using DataFrames

# Create a dataframe out of the CSV data object
data = DataFrame(data)

,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609
2,162.052,136.031,4.0612,0.0374,0.0187,116.741,-64.858,-45.216,76.96
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.449
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.648
5,51.624,21.1502,2.9085,0.242,0.134,50.8761,43.1887,9.8145,3.613
6,48.2468,17.3565,3.0332,0.2529,0.1515,8.573,38.0957,10.5868,4.792
7,26.7897,13.7595,2.5521,0.4236,0.2174,29.6339,20.456,-2.9292,0.812
8,96.2327,46.5165,4.154,0.0779,0.039,110.355,85.0486,43.1844,4.854
9,46.7619,15.1993,2.5786,0.3377,0.1913,24.7548,43.8771,-6.6812,7.875


In [47]:
# Set I to hold all of the numeric columns
I = data[:, 1:(end-1)]

,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609
2,162.052,136.031,4.0612,0.0374,0.0187,116.741,-64.858,-45.216,76.96
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.449
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.648
5,51.624,21.1502,2.9085,0.242,0.134,50.8761,43.1887,9.8145,3.613
6,48.2468,17.3565,3.0332,0.2529,0.1515,8.573,38.0957,10.5868,4.792
7,26.7897,13.7595,2.5521,0.4236,0.2174,29.6339,20.456,-2.9292,0.812
8,96.2327,46.5165,4.154,0.0779,0.039,110.355,85.0486,43.1844,4.854
9,46.7619,15.1993,2.5786,0.3377,0.1913,24.7548,43.8771,-6.6812,7.875


In [48]:
# Set O variable to hold the column of labels. "g" for "gamma" and "h" for "hadron"
O = data[:, end]

19019-element PooledArrays.PooledArray{String,UInt32,1,Array{UInt32,1}}:
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 "g"
 ⋮  
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"
 "h"

In [49]:
N = length(O) 
n = round(Int64, N / 2)

# using Pkg; Pkg.add("Random") # Uncomment and run this line if you need to install the "Random" package
using Random
R = randperm(N) # a random permutation of all the indexes (essential for sampling)
indX = R[1:n] # get some random indexes for the training set 
X = I[indX, :] # input values for training and testing set respectively
x = O[indX] # Target values for training and testing set respectively
indY = R[(n+1):end] # get some random indexes for the training set 
Y = I[indY, :] # input values for training and testing set respectively
y = O[indY] # Target values for training and testing set respectively

9509-element PooledArrays.PooledArray{String,UInt32,1,Array{UInt32,1}}:
 "g"
 "h"
 "h"
 "g"
 "g"
 "g"
 "h"
 "h"
 "g"
 "g"
 "h"
 "h"
 "g"
 ⋮  
 "h"
 "h"
 "h"
 "g"
 "g"
 "h"
 "g"
 "g"
 "h"
 "g"
 "g"
 "h"

In [50]:
# Let's test the implementation of the kNN algorithm using the Magic dataset

z = apply_kNN(X, x, Y, 5)
z

MethodError: MethodError: no method matching min(::Array{Float64,1})
Closest candidates are:
  min(::Any, !Matched::Missing) at missing.jl:104
  min(::Any, !Matched::Any) at operators.jl:414
  min(::Any, !Matched::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  ...

In [1]:
# @TODO, the inmin and inmax functions may be:
# findmin and findmax
# or minmax, possibly.